# 发现群组

### 使用feedparser解析RSS订阅源

In [1]:
# import feedparser
# import re

# # Returns title and dictionary of word counts for an RSS feed
# def getwordcounts(url:str) -> (str, list):
#   # Parse the feed
#   d = feedparser.parse(url)
#   wc = {}

#   # Loop over all the entries
#   for e in d.entries:
#     if 'summary' in e: 
#         summary=e.summary
#     else: 
#         summary=e.description

#     # Extract a list of words
#     words=getwords(e.title+' '+summary)
#     for word in words:
#       wc.setdefault(word,0)
#       wc[word]+=1
#   return d.feed.title, wc

# def getwords(html:str) -> list:
#   # Remove all the HTML tags
#   txt=re.compile(r'<[^>]+>').sub('',html)

#   # Split words by all non-alpha characters
#   words=re.compile(r'[^A-Z^a-z]+').split(txt)

#   # Convert to lowercase
#   return [word.lower() for word in words if word!='']

### 解析并准备数据

In [2]:
# apcount = {}
# wordcounts = {}
# feedlist = [line for line in open('./data/feedlist.txt')]
# for feedurl in feedlist:
#   try:
#     title, wc = getwordcounts(feedurl)
#     wordcounts[title] = wc
#     for word, count in wc.items():
#       apcount.setdefault(word, 0)
#       if count > 1:
#         apcount[word] += 1
#   except:
#     print('Failed to parse feed %s' % feedurl)

# wordlist = []
# for w, bc in apcount.items():
#   frac=float(bc)/len(feedlist)
#   if frac > 0.1 and frac < 0.5:
#     wordlist.append(w)

# out = open('./data/blogdata1.txt','w')
# out.write('Blog')
# for word in wordlist: 
#   out.write('\t%s' % word)
  
# out.write('\n')

### 读取数据

In [3]:
def readfile(filename:str)->(list, list, list):
  lines=[line for line in open(filename)]
  
  # First line is the column titles
  colnames=lines[0].strip().split('\t')[1:]
  rownames=[]
  data=[]
  for line in lines[1:]:
    p=line.strip().split('\t')
    # First column in each row is the rowname
    rownames.append(p[0])
    # The data for this row is the remainder of the row
    data.append([float(x) for x in p[1:]])
  return rownames,colnames,data

### 皮尔逊相关度

In [4]:
from math import sqrt

def pearson(v1:list, v2:list) -> float:
  # Simple sums
  sum1=sum(v1)
  sum2=sum(v2)
  
  # Sums of the squares
  sum1Sq=sum([pow(v,2) for v in v1])
  sum2Sq=sum([pow(v,2) for v in v2])	
  
  # Sum of the products
  pSum=sum([v1[i]*v2[i] for i in range(len(v1))])
  
  # Calculate r (Pearson score)
  num=pSum-(sum1*sum2/len(v1))
  den=sqrt((sum1Sq-pow(sum1,2)/len(v1))*(sum2Sq-pow(sum2,2)/len(v1)))
  if den==0: return 0

  return 1.0-num/den

### 定义聚类class

In [5]:
class bicluster:
  def __init__(self,vec,left=None,right=None,distance=0.0,id=None):
    self.left=left
    self.right=right
    self.vec=vec
    self.id=id
    self.distance=distance

### 分级聚类

In [16]:
from typing import Callable

def hcluster(rows:list, distance:Callable) -> bicluster:
  distances = {}
  currentclustid = -1

  # Clusters are initially just the rows
  clust=[bicluster(rows[i], id=i) for i in range(len(rows))]

  while len(clust)>1:
    lowestpair=(0,1)
    closest=distance(clust[0].vec,clust[1].vec)

    # loop through every pair looking for the smallest distance
    for i in range(len(clust)):
      for j in range(i+1,len(clust)):
        # distances is the cache of distance calculations
        if (clust[i].id,clust[j].id) not in distances: 
          distances[(clust[i].id,clust[j].id)]=distance(clust[i].vec,clust[j].vec)

        d=distances[(clust[i].id,clust[j].id)]

        if d<closest:
          closest=d
          lowestpair=(i,j)

    # calculate the average of the two clusters
    mergevec=[
    (clust[lowestpair[0]].vec[i]+clust[lowestpair[1]].vec[i])/2.0 
    for i in range(len(clust[0].vec))]

    # create the new cluster
    newcluster=bicluster(mergevec,left=clust[lowestpair[0]],
                         right=clust[lowestpair[1]],
                         distance=closest,id=currentclustid)

    # cluster ids that weren't in the original set are negative
    currentclustid-=1
    del clust[lowestpair[1]]
    del clust[lowestpair[0]]
    clust.append(newcluster)

  return clust[0]

In [18]:
blognames, words, data = readfile('./data/blogdata.txt')
clust = hcluster(data, pearson)
#printclust(clust, labels=blognames)

### 打印分级聚类层级结构

In [21]:
def printclust(clust:bicluster, labels=None, n=0):
  # indent to make a hierarchy layout
  for i in range(n): print (' ', end='')
  if clust.id < 0:
    # negative id means that this is branch
    print ('-')
  else:
    # positive id means that this is an endpoint
    if labels == None: print (clust.id)
    else: print (labels[clust.id])

  # now print the right and left branches
  if clust.left!=None: printclust(clust.left,labels=labels,n=n+1)
  if clust.right!=None: printclust(clust.right,labels=labels,n=n+1)

In [22]:
printclust(clust, labels=blognames)

-
 -
  Seth Godin's Blog on marketing, tribes and respect
  Autoblog
 -
  -
   SpikedHumor - Today's Videos and Pictures
   -
    -
     Captain's Quarters
     -
      TMZ.com
      PerezHilton
    -
     Steve Pavlina
     -
      -
       Joho the Blog
       BuzzMachine
      -
       Lifehack
       -
        Quick Online Tips
        -
         Copyblogger
         -
          TechCrunch
          -
           Wired
           -
            Engadget RSS Feed
            -
             -
              Search Engine Roundtable
              Google Operating System
             -
              The Official Google Blog
              -
               Schneier on Security
               -
                ShoeMoney
                -
                 mezzoblue
                 -
                  Scobleizer
                  -
                   blog maverick
                   -
                    -
                     -
                      Gothamist
                      -
        

### 绘制分级聚类的结果

In [30]:
from PIL import Image, ImageDraw

def getheight(clust):
  # Is this an endpoint? Then the height is just 1
  if clust.left==None and clust.right==None: return 1

  # Otherwise the height is the same of the heights of
  # each branch
  return getheight(clust.left)+getheight(clust.right)

def getdepth(clust):
  # The distance of an endpoint is 0.0
  if clust.left==None and clust.right==None: return 0

  # The distance of a branch is the greater of its two sides
  # plus its own distance
  return max(getdepth(clust.left),getdepth(clust.right))+clust.distance


def drawdendrogram(clust,labels,jpeg='./outputs/clusters.jpg'):
  # height and width
  h=getheight(clust)*20
  w=1200
  depth=getdepth(clust)

  # width is fixed, so scale distances accordingly
  scaling=float(w-150)/depth

  # Create a new image with a white background
  img=Image.new('RGB',(w,h),(255,255,255))
  draw=ImageDraw.Draw(img)

  draw.line((0,h/2,10,h/2),fill=(255,0,0))    

  # Draw the first node
  drawnode(draw,clust,10,(h/2),scaling,labels)
  img.save(jpeg,'JPEG')

def drawnode(draw,clust,x,y,scaling,labels):
  if clust.id<0:
    h1=getheight(clust.left)*20
    h2=getheight(clust.right)*20
    top=y-(h1+h2)/2
    bottom=y+(h1+h2)/2
    # Line length
    ll=clust.distance*scaling
    # Vertical line from this cluster to children    
    draw.line((x,top+h1/2,x,bottom-h2/2),fill=(255,0,0))    
    
    # Horizontal line to left item
    draw.line((x,top+h1/2,x+ll,top+h1/2),fill=(255,0,0))    

    # Horizontal line to right item
    draw.line((x,bottom-h2/2,x+ll,bottom-h2/2),fill=(255,0,0))        

    # Call the function to draw the left and right nodes    
    drawnode(draw,clust.left,x+ll,top+h1/2,scaling,labels)
    drawnode(draw,clust.right,x+ll,bottom-h2/2,scaling,labels)
  else:   
    # If this is an endpoint, draw the item label
    draw.text((x+5,y-7),labels[clust.id],(0,0,0))

### 图片保存至本地

In [31]:
drawdendrogram(clust, blognames, jpeg='./outputs/blogclust.jpg')

### 行和列对调，对单词聚类

In [32]:
def rotatematrix(data:list) -> list:
  newdata=[]
  for i in range(len(data[0])):
    newrow=[data[j][i] for j in range(len(data))]
    newdata.append(newrow)
  return newdata

In [34]:
rdata = rotatematrix(data)
wordclust = hcluster(rdata, pearson)
drawdendrogram(wordclust, labels=words, jpeg='./outputs/wordclust.jpg')

### k均值聚类

In [40]:
import random

def kcluster(rows, distance=pearson, k=4):

  # Determine the minimum and maximum values for each point
  ranges = [
    (min([row[i] for row in rows]), max([row[i] for row in rows])) 
    for i in range(len(rows[0]))
  ]

  # Create k randomly placed centroids
  clusters=[
    [
      random.random()*(ranges[i][1]-ranges[i][0])+ranges[i][0] 
      for i in range(len(rows[0]))
    ] 
    for _ in range(k)
  ]
  
  lastmatches = None
  for t in range(100):
    print('Iteration %d' % t)
    bestmatches=[ [] for _ in range(k) ]
    
    # Find which centroid is the closest for each row
    for j in range(len(rows)):
      row=rows[j]
      bestmatch=0
      for i in range(k):
        d=distance(clusters[i],row)
        if d<distance(clusters[bestmatch],row): bestmatch=i
      bestmatches[bestmatch].append(j)

    # If the results are the same as last time, this is complete
    if bestmatches==lastmatches: break
    lastmatches=bestmatches
    
    # Move the centroids to the average of their members
    for i in range(k):
      avgs=[0.0]*len(rows[0])
      if len(bestmatches[i])>0:
        for rowid in bestmatches[i]:
          for m in range(len(rows[rowid])):
            avgs[m]+=rows[rowid][m]
        for j in range(len(avgs)):
          avgs[j]/=len(bestmatches[i])
        clusters[i]=avgs
      
  return bestmatches

In [42]:
kclust=kcluster(data, k=4)
print(kclust)
[blognames[r] for r in kclust[1]]

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
[[5, 7, 10, 11, 12, 15, 18, 19, 21, 23, 24, 27, 38, 40, 44, 45, 47, 52, 53, 54], [8, 13, 14, 16, 20, 22, 25, 26, 28, 29, 31, 32, 33, 34, 36, 39, 49], [0, 1, 3, 4, 6, 17, 30, 35, 43, 46, 50, 51], [2, 9, 37, 41, 42, 48]]


['TechCrunch',
 'Autoblog',
 'Kotaku',
 "SpikedHumor - Today's Videos and Pictures",
 "O'Reilly Radar",
 'Joel on Software',
 'The Full Feed from HuffingtonPost.com',
 'Celebslam',
 'Lifehacker',
 'The Official Google Blog',
 'The Write News',
 'Deadspin',
 'Gizmodo',
 'Engadget RSS Feed',
 'Slashdot',
 'Copyblogger',
 'Gothamist']

## 针对偏好的聚类

In [23]:
# from BeautifulSoup import BeautifulSoup
# import urllib2
# import re
# chare=re.compile(r'[!-\.&]')
# itemowners={}

# # Words to remove
# dropwords=['a','new','some','more','my','own','the','many','other','another']

# currentuser=0
# for i in range(1,51):
#   # URL for the want search page
#   c=urllib2.urlopen(
#   'http://member.zebo.com/Main?event_key=USERSEARCH&wiowiw=wiw&keyword=car&page=%d'
#   % (i))
#   soup=BeautifulSoup(c.read())
#   for td in soup('td'):
#     # Find table cells of bgverdanasmall class
#     if ('class' in dict(td.attrs) and td['class']=='bgverdanasmall'):
#       items=[re.sub(chare,'',str(a.contents[0]).lower()).strip() for a in td('a')]
#       for item in items:
#         # Remove extra words
#         txt=' '.join([t for t in item.split(' ') if t not in dropwords])
#         if len(txt)<2: continue
#         itemowners.setdefault(txt,{})
#         itemowners[txt][currentuser]=1
#       currentuser+=1
      
# out=open('zebo.txt','w')
# out.write('Item')
# for user in range(0,currentuser): out.write('\tU%d' % user)
# out.write('\n')
# for item,owners in itemowners.items():
#   if len(owners)>10:
#     out.write(item)
#     for user in range(0,currentuser):
#       if user in owners: out.write('\t1')
#       else: out.write('\t0')
#     out.write('\n')

In [44]:
# Pearson系数适合上述博客的统治数值，而此处偏好数据只有0和1两种取值。
# 对偏好进行分析时，对同时希望拥有两件物品的人在物品方面互有重叠情况的度量是更有意义的，所以这里使用tanimoto系数，它代表的是交集与并集的比率
def tanamoto(v1:list,v2:list) -> float:
  c1,c2,shr=0,0,0
  
  for i in range(len(v1)):
    if v1[i]!=0: c1+=1 # in v1
    if v2[i]!=0: c2+=1 # in v2
    if v1[i]!=0 and v2[i]!=0: shr+=1 # in both
  
  return 1.0-(float(shr)/(c1+c2-shr))

In [48]:
wants, people, data = readfile('./data/zebo.txt')
clust = hcluster(data, distance=tanamoto)
drawdendrogram(clust, wants, jpeg='./outputs/itemclust.jpg')

In [66]:
def scaledown(data,distance=pearson,rate=0.01):
  n = len(data)

  # The real distances between every pair of items
  realdist = [
    [
      distance(data[i],data[j]) for j in range(n)
    ] 
    for i in range(0,n)
  ]
  
  # Randomly initialize the starting points of the locations in 2D
  loc = [[random.random(),random.random()] for i in range(n)]
  fakedist=[[0.0 for j in range(n)] for i in range(n)]
  
  lasterror=None
  for m in range(0,1000):
    # Find projected distances
    for i in range(n):
      for j in range(n):
        fakedist[i][j]=sqrt(
          sum([pow(loc[i][x]-loc[j][x],2) for x in range(len(loc[i]))])
        )
  
    # Move points
    grad=[[0.0,0.0] for i in range(n)]
    
    totalerror=0
    for k in range(n):
      for j in range(n):
        if j==k: continue
        # The error is percent difference between the distances
        devided_by = realdist[j][k] + 1 if realdist[j][k] == 0 else realdist[j][k] # Avoid devided by Zero
        errorterm=(fakedist[j][k]-realdist[j][k])/(devided_by) 
        
        # Each point needs to be moved away from or towards the other
        # point in proportion to how much error it has
        devided_by = fakedist[j][k] + 1 if fakedist[j][k] == 0 else fakedist[j][k] # Avoid devided by Zero
        grad[k][0]+=((loc[k][0]-loc[j][0])/fakedist[j][k])*errorterm
        grad[k][1]+=((loc[k][1]-loc[j][1])/fakedist[j][k])*errorterm

        # Keep track of the total error
        totalerror+=abs(errorterm)
    print (totalerror)

    # If the answer got worse by moving the points, we are done
    if lasterror and lasterror<totalerror: break
    lasterror=totalerror
    
    # Move each of the points by the learning rate times the gradient
    for k in range(n):
      loc[k][0]-=rate*grad[k][0]
      loc[k][1]-=rate*grad[k][1]

  return loc

def draw2d(data,labels,jpeg='mds2d.jpg'):
  img=Image.new('RGB',(2000,2000),(255,255,255))
  draw=ImageDraw.Draw(img)
  for i in range(len(data)):
    x=(data[i][0]+0.5)*1000
    y=(data[i][1]+0.5)*1000
    draw.text((x,y),labels[i],(0,0,0))
  img.save(jpeg,'JPEG')  

In [67]:
blognames, words, data = readfile('./data/blogdata.txt')
coords = scaledown(data, rate=0.01)
draw2d(coords, blognames, jpeg='./outputs/blogs2d2.jpg')

1426.508974360985
1193.4753302719791
1061.3521187452052
992.3888289626459
956.3097881612326
931.3806365102797
915.2060576996757
902.7810309579789
893.3517499689957
884.5263270705593
876.2067843786449
867.9858608190254
860.4207741547485
852.6459294384755
845.0499823122416
837.5872163470021
830.51798133836
823.6182556738404
817.7125172193718
812.0220558039597
806.5245786265278
800.6047589929608
795.1765769071549
790.479501225291
786.4835909897384
783.0336181943853
780.3297706984348
778.0562684302722
776.4720713528421
775.0221013538263
773.5182495477893
772.0148547044402
770.1807669353827
768.1788240565552
766.104742534446
764.1245253550144
762.2648916407103
760.605073724132
759.1689070897276
758.0126317969496
756.9857943840668
756.0256856191274
754.9943355927517
753.9568822362374
752.9247502742214
752.0433078333236
751.2926507919886
750.8386183695202
750.4355081208022
750.0197851856263
749.6218821481544
749.1334860130177
748.5995952268464
747.9945485703873
747.4338465812879
746.860233558